# Building the UI using Gradio
Today we will build User Interfaces using the outrageously simple Gradio framework.

Please note: your Gradio screens may appear in 'dark mode' or 'light mode' depending on your computer settings.

In [1]:
# Install if Gradio is not installed
# !pip install --upgrade gradio

In [24]:
# Importing the Libraries

import requests
import json
import time
import re
from typing import List
from IPython.display import Markdown, display, update_display
from bs4 import BeautifulSoup
import gradio as gr

In [3]:
# Setup a standard System prompt

system_message = "you are a helpful assistant"

In [4]:
OLLAMA_URL = "http://localhost:11434/api/chat"

In [16]:
def call_mistral_ollama(prompt):
    MODEL_NAME="mistral"  # Get this from you local ollama by using "ollama list" if Mistral not available do ollama run mistral
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.8,
        "stream": True  # Important: to get streaming responses
    }
    start_time = time.time()
    response = requests.post(OLLAMA_URL, json=payload, stream=True)
    # reply = response.json()["message"]["content"]
    # reply = re.sub(r"<think>.*?</think>", "", reply, flags=re.DOTALL).strip()
    # print("Total Time Taken: ", time.time()-start_time, "\n\n")
    # return reply
    result=""
    for line in response.iter_lines():
        if line:
            try:
                data = json.loads(line.decode('utf-8'))
                delta = data.get("message", {}).get("content", "")
                if delta:
                    # Optional: remove unwanted <think> tags or others
                    clean_delta = re.sub(r"<think>.*?</think>", "", delta, flags=re.DOTALL)
                    result += clean_delta
                    yield result
            except json.JSONDecodeError:
                continue

In [13]:
for data in call_mistral_ollama("what is todays date"):
    print(data)

 Today
 Today'
 Today's
 Today's date
 Today's date,
 Today's date, according
 Today's date, according to
 Today's date, according to my
 Today's date, according to my system
 Today's date, according to my system,
 Today's date, according to my system, is
 Today's date, according to my system, is March
 Today's date, according to my system, is March 
 Today's date, according to my system, is March 2
 Today's date, according to my system, is March 24
 Today's date, according to my system, is March 24,
 Today's date, according to my system, is March 24, 
 Today's date, according to my system, is March 24, 2
 Today's date, according to my system, is March 24, 20
 Today's date, according to my system, is March 24, 202
 Today's date, according to my system, is March 24, 2023
 Today's date, according to my system, is March 24, 2023.
 Today's date, according to my system, is March 24, 2023. However
 Today's date, according to my system, is March 24, 2023. However,
 Today's date, according to 

## User Interface

In [14]:
def shout(text):
    return text.upper()

In [15]:
shout("Hello")

'HELLO'

In [9]:
view = gr.Interface(fn=shout, inputs="textbox", outputs="textbox")
view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://67fe73947b7cbaacf9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
view = gr.Interface(
    fn=call_mistral_ollama,
    inputs=[gr.Textbox(label="Your Messages:", lines=8)],
    outputs=[gr.Textbox(label="Reponse:", lines=8)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [12]:
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=call_mistral_ollama,
    inputs=[gr.Textbox(label="Your Messages:")],
    outputs=[gr.Markdown(label="Reponse:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [17]:
def call_llama_ollama(prompt):
    MODEL_NAME="llama3.1"  # Get this from you local ollama by using "ollama list"
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.8,
        "stream": True  # Important: to get streaming responses
    }
    start_time = time.time()
    response = requests.post(OLLAMA_URL, json=payload, stream=True)
    # reply = response.json()["message"]["content"]
    # reply = re.sub(r"<think>.*?</think>", "", reply, flags=re.DOTALL).strip()
    # print("Total Time Taken: ", time.time()-start_time, "\n\n")
    # return reply
    result=""
    for line in response.iter_lines():
        if line:
            try:
                data = json.loads(line.decode('utf-8'))
                delta = data.get("message", {}).get("content", "")
                if delta:
                    # Optional: remove unwanted <think> tags or others
                    clean_delta = re.sub(r"<think>.*?</think>", "", delta, flags=re.DOTALL)
                    result += clean_delta
                    yield result
            except json.JSONDecodeError:
                continue

In [14]:
view = gr.Interface(
    fn=call_llama_ollama,
    inputs=[gr.Textbox(label="Your Messages:")],
    outputs=[gr.Markdown(label="Reponse:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [18]:
def select_model(prompt, model):
    if model == "Llama":
        result = call_llama_ollama(prompt)
    elif model == "Mistral":
        result = call_mistral_ollama(prompt)
    else:
        raise ValueError("Unknown Model")
    yield from result

In [19]:
view = gr.Interface(
    fn=select_model,
    inputs=[gr.Textbox(label="Your Messages:"), gr.Dropdown(["Llama", "Mistral"], label="Select Model")],
    outputs=[gr.Markdown(label="Reponse:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


# Building a company brochure generator

In [20]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """
    def __init__(self, url):
        self.url = url
        response = requests.get(headers=headers, url=self.url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No Title Found"
        if soup.body:
            for irrelevent in soup.body(["script", "style", "img", "input"]):
                irrelevent.decompose()
            self.text = soup.body.get_text(separator='\n', strip=True)
        else:
            self.text = ""

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [21]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [22]:
def stream_brochure(company_name, url, model):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt+=Website(url).get_contents()
    if model == "Llama":
        result = call_llama_ollama(prompt)
    elif model == "Mistral":
        result = call_mistral_ollama(prompt)
    else:
        raise ValueError("Unknown Model")
    yield from result

In [27]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company Name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["Llama", "Mistral"], label="Select Model")
    ],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never",
)
view.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
